# 예제: 특정 도메인 웹사이트에서 웹 크롤링과 전처리, 그리고 벡터화 하기

도메인에 특화된 코퍼스를 확보하는 것은 언어 인식 데이터를 만드는 데 필수적입니다.

우리의 언어 모델에 충분한 훈련 예시를 제공하려면 매우 큰 코퍼스가 필요합니다.

도메인의 특이성을 높이면 말뭉치의 양을 줄이는 것은 시간 문제입니다.

지금까지 배운 것을 응용하여 관심있는 도메인에 특화된 웹사이트에서 텍스트 데이터를 수집하여 전처리하고 벡터화 해봅시다!

현재 디렉토리의 경로를 지정합니다.

In [ ]:
%cd /content
# 세션 저장소에서 현재 디렉토리의 경로입니다.
# 구글 드라이브를 마운팅하여 구글 드라이브를 저장공간으로 사용할 수 있습니다. 자세한 내용은 메뉴 'Help' > 'Search code snippets' > 'mount' 검색

먼저, 앞서 배운 것을 응용하여 HTML로 텍스트 데이터를 수집하는 함수를 정의해 봅시다!

사용할 라이브러리를 설치하려면 다음 명령을 실행하세요.

In [ ]:
# 앞서 배운 내용을 참고하여 사용할 라이브러리를 설치해 보세요.

사용할 라이브러리들을 불러옵니다.

In [1]:
# 라이브러리들을 불러와 보세요.
import os
import bs4
import requests
import feedparser
import re
import string
import time
import nltk
nltk.download('punkt')
from slugify import slugify
from nltk import sent_tokenize
from nltk.corpus.reader.api import CorpusReader
from nltk.corpus.reader.api import CategorizedCorpusReader
from readability.readability import Unparseable
from readability.readability import Document

[nltk_data] Downloading package punkt to /home/ljj0512/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


관심있는 도메인을 정하여 문자열인 도메인 이름을 할당하여 변수 `domain_name`를 선언합니다.

변수 `domain_name`를 사용하여 지정된 경로의 저장공간에 HTML을 저장할 폴더를 만듭니다.

해당 도메인에 특화된 웹사이트를 선택하여 수집할 웹사이트 URL 리스트를 할당하여 변수 `sources`를 선언합니다.

웹사이트에서 얻은 HTML을 텍스트로 바꿔 저장하는 함수를 정의합니다.

In [2]:
############## meta data #############
domain_name = "Finance" # 관심있는 도메인의 이름
sources = [
    "https://www.koreatimes.co.kr/www/section_602.html",
    "http://www.koreaherald.com/list.php?ct=021900000000",
    "http://www.theinvestor.co.kr/",
] # 관심있는 도메인에 특화된 여러 웹사이트 URL
######################################

domain_folder_path = os.getcwd() + '/' + domain_name

if not os.path.exists(domain_name):
    os.mkdir(domain_name)

TAGS = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'p', 'li']

site_name_list = [] # 웹사이트 이름들을 리스트로 할당하기 위해 빈 리스트를 만듭니다.
path_list = [] # 파일 경로들을 리스트로 할당하기 위해 빈 리스트를 만듭니다.

def crawl_html_to_text(url):
    # 웹사이트에서 HTML을 가져오고 그것을 텍스트로 바꿔 저장하는 함수를 정의해 보세요.
    global domain_folder_path, domain_name, site_name_list, path_list
    
    # site의 domain, resource name 추출
    tmp = url.split("//")
    protocol = tmp[0]   # protocol 저장 (e.g. https:)
    site_name = tmp[-1].split("/")[0].split(".")[1]   # domain name of the site 저장 (e.g. google)
    site_resource = tmp[-1].split("/")[0]   # resource name of the site 저장 (e.g. www.google.com)
    site_name_list.append(site_name)

    # site 이름으로 folder를 만듬, html로 txt폴더 나눔
    if not os.path.exists(f"./{domain_name}/{site_name}"):
        os.mkdir(f"./{domain_name}/{site_name}")
    if not os.path.exists(f"./{domain_name}/{site_name}/html"):
        os.mkdir(f"./{domain_name}/{site_name}/html")
    if not os.path.exists(f"./{domain_name}/{site_name}/txt"):
        os.mkdir(f"./{domain_name}/{site_name}/txt")

    # site에서 html 가져온 후 parsing
    html = requests.get(url).content
    soup = bs4.BeautifulSoup(html, "lxml")
    links = set(soup.find_all('a',href=True))

    # 값 준비
    max_sub_urls = 5    # 기본 url에서 sub url을 이 값에 따라 추출
    sub_urls = [url]
    filename_list = ['main']

    # sub_url 값 추출
    for i, link in enumerate(links):
        if i == max_sub_urls:
            break
        # 앞 http또는 https가 들어가있는 hyper link는 사이트의 내용과는 관계없는 경우가 많아 이를 제외
        if not re.search('(https?:)', link['href']):
            sub_urls.append(f"{protocol}//{site_resource}{link['href']}")
            if link.string == None:
                filename_list.append(
                    slugify(str(link.string)+str(i)).lower()
                )
            else:
                filename_list.append(
                    slugify(str(link.string)).lower()
                )
    
    # 추출한 url에서 html을 가져와 txt파일로 저장
    for sub_url, filename in zip(sub_urls, filename_list):
        try:
            # print(sub_url)
            page = requests.get(sub_url).content
            path = domain_folder_path + "/" + site_name
            path_list.append(path)
            # store origin html file in html folder
            path = domain_folder_path + "/" + site_name + "/" + "html" + "/" + filename + ".html"
            with open(path, "wb") as f:
                f.write(page)
            # store txt file in txt folder
            path = domain_folder_path + "/" + site_name + "/" + "txt" + "/" + filename + ".txt"
            with open(path, "wb") as f:
                f.write(page)
        except:
            print(f"********* Error occur at ({sub_url}) *********")

저장한 파일의 문서에서 문단을 분리하고 전처리하여 각 문단에 대한 기술 통계를 확인하는 함수를 정의합니다.

In [3]:
CAT_PATTERN = r'([a-z_\s]+)/.*' #r'([a-z_\s]+)/.*'
DOC_PATTERN = r'.+\.html'
TAGS = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'li']

class HTMLCorpusReader(CategorizedCorpusReader, CorpusReader):
    """
    A corpus reader for raw HTML documents to enable preprocessing.
    """

    def __init__(self, root, fileids=DOC_PATTERN, encoding='utf8',
                 tags=TAGS, **kwargs):
        # print('root:',root)
        # print('fileids:',fileids)
        # print('tags:',tags)

        """
        Initialize the corpus reader.  Categorization arguments
        (``cat_pattern``, ``cat_map``, and ``cat_file``) are passed to
        the ``CategorizedCorpusReader`` constructor.  The remaining
        arguments are passed to the ``CorpusReader`` constructor.
        """
        # Add the default category pattern if not passed into the class.
        if not any(key.startswith('cat_') for key in kwargs.keys()):
            kwargs['cat_pattern'] = CAT_PATTERN
        
        # print('kwargs:',kwargs)
        
        # Initialize the NLTK corpus reader objects
        CategorizedCorpusReader.__init__(self, kwargs)
        CorpusReader.__init__(self, root, fileids, encoding)

        # Save the tags that we specifically want to extract.
        self.tags = tags

    def resolve(self, fileids, categories):
        """
        Returns a list of fileids or categories depending on what is passed
        to each internal corpus reader function. Implemented similarly to
        the NLTK ``CategorizedPlaintextCorpusReader``.
        """
        if fileids is not None and categories is not None:
            raise ValueError("Specify fileids or categories, not both")

        if categories is not None:
            return self.fileids(categories)
        return fileids

    def docs(self, fileids=None, categories=None):
        """
        Returns the complete text of an HTML document, closing the document
        after we are done reading it and yielding it in a memory safe fashion.
        """
        # Resolve the fileids and the categories
        fileids = self.resolve(fileids, categories)

        # Create a generator, loading one document into memory at a time.
        for path in self.abspaths(fileids):
            with open(path, 'r') as f:
                yield f.read()

    def sizes(self, fileids=None, categories=None):
        """
        Returns a list of tuples, the fileid and size on disk of the file.
        This function is used to detect oddly large files in the corpus.
        """
        # Resolve the fileids and the categories
        fileids = self.resolve(fileids, categories)

        # Create a generator, getting every path and computing filesize
        for path in self.abspaths(fileids):
            yield os.path.getsize(path)


In [4]:
def paras(docs):
    # 문서에서 문단을 분리하는 함수를 정의해 보세요.
    
    # 필요없는 테그들의 내용들을 없앤다.
    def gener(docs):
        for doc in docs:
            try:
                yield Document(doc).summary()
            except Unparseable as e:
                print("Could not parse HTML: {}".format(e))
                continue
    docs = list(gener(docs))
    print(len(docs))
    
    # tags에 해당하는 tag 안에 내용을 추출
    tags = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'p', 'li']
    for doc in docs:
        soup = bs4.BeautifulSoup(doc, 'lxml')
        for element in soup.find_all(tags):
            yield element.text
        soup.decompose()    



def describe(paragraphs, corpus):
    started = time.time() # 현 시점의 시간을 할당합니다.
    # 앞서 배운 내용을 참고하여 각 문단에 대한 기술 통계를 확인하는 함수를 정의해 보세요.

    counts = nltk.FreqDist()
    tokens = nltk.FreqDist()

    for para in paragraphs:
        counts['paras'] += 1

        for sent in nltk.sent_tokenize(para):
            counts['sents'] += 1

            for word in nltk.wordpunct_tokenize(sent):
                counts['words'] += 1
                tokens[word] += 1
    
    n_fileids = len(corpus.fileids())
    n_topics = len(corpus.categories())

    return {
        '# of files': n_fileids,
        '# of topics': n_topics,
        '# of paragraphs': counts['paras'],
        '# of sentences': counts['sents'],
        '# of words': counts['words'],
        'vocabulary size': len(tokens), # 중복 단어 제거 시 실 언휘 수
        'lexical diversity': float(counts['words']) / float(len(tokens)),
        'paragraphs per document':  float(counts['paras']) / float(n_fileids),
        'sentences per paragraph':  float(counts['sents']) / float(counts['paras']),
        'secs':   time.time() - started,
    }

그런 다음, 텍스트 데이터를 TF-IDF 인코딩으로 벡터화하는 함수를 정의해 봅시다!

다양한 기계 학습 알고리즘을 구현하는 `sklearn` 라이브러리를 사용하여 기계 학습을 수행하는 데 한 발짝 더 다가가 봅시다.

In [5]:
# TF-IDF 인코딩을 제공하는 sklean 라이브러리의 메서드를 불러와 보세요.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

`sklearn` 라이브러리를 사용하여 문서를 벡터화하는 함수를 정의합니다.

In [6]:
def tfidf_vectorize(paragraphs):
    # sklean 라이브러리의 메서드를 사용하여 문서를 벡터화하는 함수를 정의해 보세요.
    tfidf = TfidfVectorizer()
    return tfidf.fit_transform(paragraphs)#tfidf.get_feature_names_out()

이제 앞에서 정의한 함수들을 실행하여 문서를 구문 분석하고 벡터화한 결과를 확인해 봅시다!

In [7]:
# for 반복문을 사용하여 웹사이트 URL을 하나씩 받아 HTML을 가져오고 텍스트로 바꿔 저장해 봅시다.
# print()를 사용하여 관심있는 도메인의 이름, 파일 아이디 개수, 크롤링한 웹사이트 개수 등을 프린팅해 봅시다.
# for 반복문을 사용하여 저장한 문서를 하나씩 받아 구조화 한 다음, 웹사이트 이름과 분리된 각 문단에 대한 기술 통계 및 벡터 값을 프린팅해 봅시다.
for url in sources:
    crawl_html_to_text(url)

********* Error occur at (http://www.theinvestor.co.kr/view.php?ud=20220919000080) *********


In [21]:
print(len(site_name_list))
print(site_name_list)
print()
print(len(path_list))
print(path_list)

3
['koreatimes', 'koreaherald', 'theinvestor']

16
['/home/ljj0512/private/data-mining/assign01/Finance/koreatimes', '/home/ljj0512/private/data-mining/assign01/Finance/koreatimes', '/home/ljj0512/private/data-mining/assign01/Finance/koreatimes', '/home/ljj0512/private/data-mining/assign01/Finance/koreatimes', '/home/ljj0512/private/data-mining/assign01/Finance/koreatimes', '/home/ljj0512/private/data-mining/assign01/Finance/koreatimes', '/home/ljj0512/private/data-mining/assign01/Finance/koreaherald', '/home/ljj0512/private/data-mining/assign01/Finance/koreaherald', '/home/ljj0512/private/data-mining/assign01/Finance/koreaherald', '/home/ljj0512/private/data-mining/assign01/Finance/koreaherald', '/home/ljj0512/private/data-mining/assign01/Finance/theinvestor', '/home/ljj0512/private/data-mining/assign01/Finance/theinvestor', '/home/ljj0512/private/data-mining/assign01/Finance/theinvestor', '/home/ljj0512/private/data-mining/assign01/Finance/theinvestor', '/home/ljj0512/private/data-mi

In [19]:
corpus = HTMLCorpusReader(domain_name, fileids=DOC_PATTERN, tags=TAGS, cat_pattern=CAT_PATTERN)
print(len(corpus.categories()))
print((corpus.categories()))
print()
print(len(corpus.fileids()))
print((corpus.fileids()))

3
['koreaherald', 'koreatimes', 'theinvestor']

15
['koreaherald/html/diplomatic-circuit.html', 'koreaherald/html/hangugeopan.html', 'koreaherald/html/main.html', 'koreaherald/html/social-affairs.html', 'koreatimes/html/blogs.html', 'koreatimes/html/books.html', 'koreatimes/html/busan-s-drive-for-english-as-common-language-rolls-on-despite-protests.html', 'koreatimes/html/business.html', 'koreatimes/html/main.html', 'koreatimes/html/markets.html', 'theinvestor/html/about-herald-corporation.html', 'theinvestor/html/deals.html', 'theinvestor/html/main.html', 'theinvestor/html/none2.html', 'theinvestor/html/none4.html']


In [10]:
docs = corpus.docs(categories=corpus.categories()[0])
paragraphs = list(paras(docs))
for para in paragraphs:
    print(para)
    print("*"*100)



4
[Herald Interview]Korea’s hard, soft power offers robust room for cooperation: Malaysian ambassador
****************************************************************************************************
Sept 13, 2022
****************************************************************************************************
Korea is a role model for Malaysia in its development of soft and hard power, and the two countries’ relationship has great room for cooperation, said Malaysian Ambassador Lim Juay Jin during a recent interview with The Korea Herald. Citing Keluarga Malaysia, an aspiration of the Malaysian government to forge unity and brotherhood to achieve development, and Korea’s national unity concept, Lim hoped to bring together the aspirations of Malaysia and Korea. “Malaysia aspires to b
****************************************************************************************************
[단독] ‘이춘재 연쇄살인 누명’ 윤성여, “‘검수완박’ 되면 억울한 사람 늘어날 것”
*************************************************

In [11]:
for para in paragraphs:
    for sent in nltk.sent_tokenize(para):
        print(sent)
        print("*"*50)

[Herald Interview]Korea’s hard, soft power offers robust room for cooperation: Malaysian ambassador
**************************************************
Sept 13, 2022
**************************************************
Korea is a role model for Malaysia in its development of soft and hard power, and the two countries’ relationship has great room for cooperation, said Malaysian Ambassador Lim Juay Jin during a recent interview with The Korea Herald.
**************************************************
Citing Keluarga Malaysia, an aspiration of the Malaysian government to forge unity and brotherhood to achieve development, and Korea’s national unity concept, Lim hoped to bring together the aspirations of Malaysia and Korea.
**************************************************
“Malaysia aspires to b
**************************************************
[단독] ‘이춘재 연쇄살인 누명’ 윤성여, “‘검수완박’ 되면 억울한 사람 늘어날 것”
**************************************************
Apr 18, 2022
***********************************

In [12]:
for para in paragraphs:
    for sent in nltk.sent_tokenize(para):
        for word in nltk.word_tokenize(sent):
            print(word)
            print("*"*50)

[
**************************************************
Herald
**************************************************
Interview
**************************************************
]
**************************************************
Korea
**************************************************
’
**************************************************
s
**************************************************
hard
**************************************************
,
**************************************************
soft
**************************************************
power
**************************************************
offers
**************************************************
robust
**************************************************
room
**************************************************
for
**************************************************
cooperation
**************************************************
:
**************************************************
Malaysian
*************************************

In [13]:
describe(paragraphs, corpus)

{'# of files': 15,
 '# of topics': 3,
 '# of paragraphs': 12,
 '# of sentences': 23,
 '# of words': 525,
 'vocabulary size': 301,
 'lexical diversity': 1.744186046511628,
 'paragraphs per document': 0.8,
 'sentences per paragraph': 1.9166666666666667,
 'secs': 0.005862712860107422}

In [14]:
tfidf = tfidf_vectorize(paragraphs)
print(tfidf)

  (0, 32)	0.2772555156944645
  (0, 114)	0.2772555156944645
  (0, 56)	0.2772555156944645
  (0, 76)	0.2449154749259529
  (0, 148)	0.2772555156944645
  (0, 146)	0.3228362431088391
  (0, 126)	0.3228362431088391
  (0, 135)	0.2772555156944645
  (0, 160)	0.2772555156944645
  (0, 84)	0.2772555156944645
  (0, 103)	0.1993347475115783
  (0, 95)	0.2772555156944645
  (0, 86)	0.2772555156944645
  (1, 8)	0.4589581693061651
  (1, 2)	0.6740114321865919
  (1, 153)	0.578848847376336
  (2, 41)	0.09315835947579601
  (2, 40)	0.09315835947579601
  (2, 173)	0.09315835947579601
  (2, 45)	0.09315835947579601
  (2, 90)	0.09315835947579601
  (2, 53)	0.09315835947579601
  (2, 123)	0.09315835947579601
  (2, 27)	0.09315835947579601
  (2, 47)	0.09315835947579601
  :	:
  (11, 21)	0.07281699192264762
  (11, 43)	0.07281699192264762
  (11, 50)	0.07281699192264762
  (11, 60)	0.14563398384529525
  (11, 124)	0.14563398384529525
  (11, 100)	0.07281699192264762
  (11, 74)	0.07281699192264762
  (11, 63)	0.14563398384529525
  (

In [141]:
from nltk.text import TextCollection

def tokenize(text):
    stem = nltk.stem.SnowballStemmer('english')
    text = text.lower()

    for token in nltk.word_tokenize(text):
        if token in string.punctuation: continue
        yield stem.stem(token)

def nltk_tfidf_vectorize(paragraphs):

    tokenized_corpus = [list(tokenize(doc)) for doc in paragraphs]
    texts = TextCollection(tokenized_corpus)

    for doc in tokenized_corpus:
        yield {
            term: texts.tf_idf(term, doc)
            for term in doc
        }

for toks in nltk_tfidf_vectorize(paragraphs):
    print(toks)
    print()

{'s.': 0.2233269391516403, 'korea': 0.10967706633574531, 'us': 0.10967706633574531, 'to': 0.06050380346036002, 'discuss': 0.26953675118897, '‘': 0.3925918638888587, 'concret': 0.26953675118897, '’': 0.3001722398141994, 'stronger': 0.2233269391516403, 'measur': 0.2233269391516403, 'for': 0.16224089036002998, 'n.korean': 0.26953675118897, 'threat': 0.2233269391516403}

{'sept': 1.1166346957582016, '14': 1.34768375594485, '2022': 0.8112044518001499}

{'us': 0.10282224968976122, 'nuclear-pow': 0.2526907042396594, 'supercarri': 0.2526907042396594, 'will': 0.11536416815614567, 'stage': 0.2526907042396594, 'militari': 0.2526907042396594, 'drill': 0.2526907042396594, 'with': 0.1310713199237023, 'south': 0.10877913592753154, 'korean': 0.14070573741290596, 'warship': 0.2526907042396594, 'for': 0.1521008347125281, 'first': 0.18402743619790252, 'time': 0.18402743619790252, 'sinc': 0.2526907042396594, '2017': 0.2526907042396594}

{'단독': 0.23782654516673823, '‘': 0.34640458578428707, '이춘재': 0.197053

In [15]:
tfidf = TfidfVectorizer()
print(len(tfidf.fit_transform(paragraphs).toarray()[0]))
print(len(tfidf.get_feature_names_out()))

266
266


수집한 텍스트 데이터는 도메인에 대해 어떤 이야기를 들려 주고 있나요?

지금까지 배운 것들을 인터넷에 있는 수많은 텍스트 데이터를 언어 모델로 분석하는 데 사용할 수 있습니다!

> Reference: <br>
https://docs.github.com/en/repositories/creating-and-managing-repositories/cloning-a-repository <br>
https://github.com/wilkens-teaching/textmining <br>
https://github.com/foxbook/atap </br>